In [1]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6949896836421732074
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14444920832
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14869481660691502597
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
!pip install -q datasets
!pip install -q  transformers
!pip install -q seqeval[cpu]
!pip install -q -U spacy
!python -q -m spacy download en_core_web_sm
#!pip install -q allennlp==1.0.0 allennlp-models==1.0.0
#!pip install -q gensim

     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 65.7 MB/s 
     |████████████████████████████████| 115 kB 68.3 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 141 kB 69.1 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
     |████████████████████████████████| 127 kB 73.3 MB/s 
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 55.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import logging
import pandas as pd
from datasets import Dataset
from collections import Counter, defaultdict
from time import time
import string
from tqdm import tqdm
import spacy
import gensim.downloader as api

from seqeval.metrics import f1_score, classification_report
from sklearn.metrics import classification_report as sk_classification_report
from sklearn.metrics import f1_score as sk_f1_score

# from allennlp.predictors.predictor import Predictor
# import allennlp_models.structured_prediction

In [4]:
nlp = spacy.load("en_core_web_sm")
#word_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
device = 'cuda'
model_name = 'bert-large-cased'
model = BertForMaskedLM.from_pretrained(model_name).to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Dataset to Evaluate on (in CoNLL Format)

In [6]:
!git clone https://github.com/uds-lsv/TOKEN-is-a-MASK


Cloning into 'TOKEN-is-a-MASK'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21 (delta 1), reused 18 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [7]:
data_path = 'TOKEN-is-a-MASK/data/conll03NER/'

In [8]:
def get_dataset(file):
  df = pd.read_csv(file, sep=" ", header=None, usecols=[0, 3], skip_blank_lines=False)
  df.columns = ['token', 'tag']
  df['tag'] = df['tag'].fillna('O')
  #df.tag[df.tag == 'B-MISC'] = 'O'
  #df.tag[df.tag == 'I-MISC'] = 'O'
  dataset = Dataset.from_pandas(df)

  return dataset
#train_dataset = get_dataset('train.txt')
#dev_dataset = get_dataset('dev.txt')
test_dataset = get_dataset(data_path+'test.txt')

# Using POS 

In [9]:
ent_pos = 'PROPN'
def get_pos(seq):
  text = " ".join(seq)
  doc = nlp(text)
  token_tags = []
  for token in doc:
    if token.pos_ == 'SPACE':
      token_tags.append([' ', token.pos_])
    elif len(token) == 1:
      token_tags.append([token.text, 'O'])
    else:
      token_tags.append([token.text, token.pos_])
  
  seq_tags = []
  counter = 0
  running_token = ''
  running_tags = []
  for t, pos in token_tags:
    if t == seq[counter]:
      seq_tags.append([t, pos])
      counter += 1
    else:
      running_token += t
      running_tags += [pos]
      if running_token == seq[counter]:
        if ent_pos in running_tags:
          _pos = ent_pos
        else:
          _pos = running_tags[0]
        seq_tags.append([running_token, _pos])
        counter += 1
        running_token = ''
        running_tags = []

  assert seq == [t for t, _ in seq_tags]

  for i in range(len(seq)):
    if seq_tags[i][1] == ent_pos:
      seq_tags[i][0] = seq_tags[i][0].capitalize()
      seq[i] = seq[i].capitalize()
  seq[0] = seq[0].capitalize()
  return seq, seq_tags

In [10]:
i = 16
j = 22
seq = test_dataset[i: j]['token']
get_pos(seq)

(['Al-ain', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 [['Al-ain', 'PROPN'],
  [',', 'O'],
  ['United', 'PROPN'],
  ['Arab', 'PROPN'],
  ['Emirates', 'PROPN'],
  ['1996-12-06', 'NUM']])

In [11]:
def get_labels(text, template_text):
  tokenized_text = tokenizer.tokenize(text)
  tokenized_template = tokenizer.tokenize(template_text)
  _tokenized_text = tokenized_text + tokenized_template
  masked_ids = [i for i, x in enumerate(_tokenized_text) if x == "[MASK]"]
  indexed_tokens = tokenizer.convert_tokens_to_ids(_tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens]).to(device)

  segments_ids_text = [0] * len(tokenized_text)
  segments_ids_template = [1] * len(tokenized_template)
  segments_ids = segments_ids_text + segments_ids_template
  segments_tensors = torch.tensor([segments_ids]).to(device)

  outputs = model(tokens_tensor, token_type_ids=segments_tensors)
  predictions = outputs[0]
  prob = torch.softmax(predictions[0, masked_ids], dim=-1)

  labels_dic = {}
  for tag, ids in ner_words.items():
      _max_prob, _max_id = torch.max(prob[0, ids], 0)
      labels_dic[tag] = (_max_prob.item(), ner_words_text[tag][_max_id.item()])
      #labels_dic[tag] = prob[0, ids].max().item()
  pred_label = max(labels_dic, key=labels_dic.get)
  pred_prob, pred_word  = labels_dic[pred_label]
  return pred_label, pred_prob, pred_word

In [12]:
def classifier(seq, pos_tags):
  text_seq = ["[CLS]"] + seq + ["[SEP]"]
  text =  ' '.join(text_seq)

  len_pos = len(pos_tags)
  pred_tags = []
  pred_words = []
  _entity = '' 
  counter = 0
  for i, (t, pos) in enumerate(pos_tags):
    if pos != ent_pos:
      pred_tags.append('O')
      pred_words.append('None')
      _entity = ''
      counter = 0
      continue

    _entity += ' ' + t
    counter += 1
    if i+1 < len_pos and pos_tags[i+1][1] == ent_pos:
      continue
    else:
      pred_tag = None
      pred_prob = 0
      pred_word = None
      for template in templates:
        template_text = _entity +  template
        _pred_label, _pred_prob, _pred_word = get_labels(text, template_text)
        if _pred_prob > pred_prob:
          pred_prob = _pred_prob
          pred_tag = _pred_label
          pred_word = _pred_word 
      pred_tags += [pred_tag] * counter
      pred_words += [pred_word] * counter

  assert len(pred_tags) == len(seq), print(seq, len(pred_tags), len(seq))

  bio_pred_tags = []
  for i, tag in enumerate(pred_tags):
    if tag == 'O':
      bio_pred_tags += [tag]
      continue
    if len(bio_pred_tags) == 0:
      bio_pred_tags += ['B-'+tag]

    elif tag in bio_pred_tags[i-1]:
      bio_pred_tags += ['I-'+tag]
    else:
      bio_pred_tags += ['B-'+tag]

  return bio_pred_tags, pred_words

In [13]:
def flatten(_list):
  return [item for sublist in _list for item in sublist]

def compute_bare_f1(ner_tags, pred_tags):
  bare_ner_tages = [tag[2:] if tag != 'O' else tag for tag in ner_tags]
  bare_pred_tages = [tag[2:] if tag != 'O' else tag for tag in pred_tags]
  print(sk_classification_report(bare_ner_tages, bare_pred_tages))
  return sk_f1_score(bare_ner_tages, bare_pred_tages, average=None)

def evaluation(dataset, f1_steps):
  seq = []
  ner_tags = []
  total_ner_tags = []
  total_pred_tags = []
  total_pred_words = []
  num_samples = 0
  for i, token in tqdm(enumerate(dataset['token'])):
    tag = dataset['tag'][i]
    if token == None:
      num_samples += 1
      add_point = False
      if seq[-1] != '.':
          add_point = True
          seq += ['.']
      seq, pos_tags = get_pos(seq)
      pred_tags, pred_words = classifier(seq, pos_tags) 
      if add_point:
        pred_tags = pred_tags[:-1]
        pred_words = pred_words[:-1]
      assert len(pred_tags) == len(ner_tags), len(pred_words) == len(ner_tags)
      total_ner_tags.append(ner_tags)
      total_pred_tags.append(pred_tags)
      total_pred_words.append(pred_words)
      if num_samples % f1_steps == 0:
        #compute_bare_f1(flatten(total_ner_tags), flatten(total_pred_tags))
        print(classification_report(total_ner_tags, total_pred_tags))
        print('---------------')
      seq = []
      ner_tags = []
    else:
      seq += [token]
      ner_tags += [tag]

  #compute_bare_f1(flatten(total_ner_tags), flatten(total_pred_tags))
  print(classification_report(total_ner_tags, total_pred_tags))
  return total_ner_tags, total_pred_tags, total_pred_words

In [14]:
def get_word_ids(words):
  ids = []
  for w in words:
    _id = tokenizer.convert_tokens_to_ids(w)
    ids.append(_id)
  
  assert len(ids) == len(words)
  return ids

init_loc_words = ['location', 'city', 'country', 'region', 'area',
             'province', 'state', 'town', 'downtown']
init_per_words = ['person', 'man', 'woman', 'boy', 'girl', 'human', 'someone', 'kid']
init_org_words = ['organization', 'community', 'commision', 'department',
             'association', 'council', 'company', 'union', 'team', 'group', 'firm']
init_misc_words = ['citizenship', 'nationality', 'ethnicity', 'cup' , 'game']

ner_words_text = {'ORG': init_org_words, #'MISC': init_misc_words,
            'PER': init_per_words, 'LOC': init_loc_words}

ner_words = {k: get_word_ids(v) for k, v in ner_words_text.items()}   

templates = [' is a [MASK] . [SEP]', ' is an [MASK] . [SEP]']
#templates = [' is an instance of a [MASK] . [SEP]', ' is an instance of an [MASK] . [SEP]']

#templates = [' denotes a [MASK] . [SEP]', ' denotes an [MASK] . [SEP]']

#templates = ['  would be a [MASK] . [SEP]', '  would be an [MASK] . [SEP]']

#templates = ['  is well known to be  a [MASK] . [SEP]', '  is well known to be an  [MASK] . [SEP]']

#templates = ['   is a common [MASK] known to many people. [SEP]']


#templates = [' [MASK] . [SEP]', ' [MASK] . [SEP]']

ner_words

{'ORG': [2369, 1661, 100, 2853, 3852, 3193, 1419, 3779, 1264, 1372, 3016],
 'PER': [1825, 1299, 1590, 2298, 1873, 1769, 1800, 5102],
 'LOC': [2450, 1331, 1583, 1805, 1298, 3199, 1352, 1411, 5215]}

In [15]:
gt_labels, pred_labels, pred_words = evaluation(test_dataset, f1_steps=200)
#evaluation(test_dataset[0:49888], f1_steps=200)

2774it [02:09, 28.13it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seq

              precision    recall  f1-score   support

         LOC       0.80      0.78      0.79       141
        MISC       0.00      0.00      0.00        52
           O       0.98      0.97      0.98      1982
         ORG       0.27      0.57      0.37        67
         PER       0.95      0.97      0.96       333

    accuracy                           0.93      2575
   macro avg       0.60      0.66      0.62      2575
weighted avg       0.93      0.93      0.93      2575

              precision    recall  f1-score   support

         LOC       0.80      0.78      0.79       130
        MISC       0.00      0.00      0.00        35
         ORG       0.26      0.51      0.34        49
         PER       0.91      0.90      0.90       193

   micro avg       0.72      0.73      0.73       407
   macro avg       0.49      0.55      0.51       407
weighted avg       0.71      0.73      0.72       407

---------------


5531it [04:16, 17.88it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seq

              precision    recall  f1-score   support

         LOC       0.70      0.68      0.69       312
        MISC       0.00      0.00      0.00       110
           O       0.98      0.96      0.97      3819
         ORG       0.19      0.42      0.26       137
         PER       0.95      0.97      0.96       756

    accuracy                           0.91      5134
   macro avg       0.56      0.61      0.58      5134
weighted avg       0.92      0.91      0.91      5134

              precision    recall  f1-score   support

         LOC       0.69      0.69      0.69       266
        MISC       0.00      0.00      0.00        63
         ORG       0.18      0.36      0.24       102
         PER       0.91      0.93      0.92       425

   micro avg       0.68      0.72      0.70       856
   macro avg       0.45      0.50      0.46       856
weighted avg       0.69      0.72      0.70       856

---------------


7274it [05:48, 15.45it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seq

...

              precision    recall  f1-score   support

         LOC       0.61      0.81      0.70      1207
        MISC       0.00      0.00      0.00       545
           O       0.98      0.96      0.97     24972
         ORG       0.51      0.63      0.56      1433
         PER       0.80      0.96      0.87      1613

    accuracy                           0.92     29770
   macro avg       0.58      0.67      0.62     29770
weighted avg       0.91      0.92      0.92     29770



/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
31773it [22:47, 10.65it/s]

              precision    recall  f1-score   support

         LOC       0.60      0.80      0.69      1021
        MISC       0.00      0.00      0.00       433
         ORG       0.39      0.49      0.43       901
         PER       0.71      0.84      0.77       975

   micro avg       0.57      0.62      0.60      3330
   macro avg       0.43      0.53      0.47      3330
weighted avg       0.50      0.62      0.55      3330

---------------


33720it [24:04, 23.35it/s]


AssertionError: ignored